In [1]:
import pickle
import os
import time
import numpy as np
import polychrom

from polychrom import polymerutils
from polychrom import forces
from polychrom import forcekits
from polychrom.simulation import Simulation
from polychrom.starting_conformations import grow_cubic
from polychrom.hdf5_format import HDF5Reporter, list_URIs, load_URI, load_hdf5_file

import simtk.openmm 
import os 
import shutil


import warnings
import h5py 
import glob

## Defining simulation parameters

In [2]:
# -------defining parameters----------
#  -- basic loop extrusion parameters

folder = "trajectory"

myfile = h5py.File("trajectory/LEFPositions.h5", mode='r')

N = myfile.attrs["N"]
LEFNum = myfile.attrs["LEFNum"]
LEFpositions = myfile["positions"]

Nframes = LEFpositions.shape[0]

    
steps = 750   # MD steps per step of cohesin
stiff = 1
dens = 0.1
box = (N / dens) ** 0.33  # density = 0.1.
data = grow_cubic(N, int(box) - 2)  # creates a compact conformation 
block = 0  # starting block 

# new parameters because some things changed 
saveEveryBlocks = 10   # save every 10 blocks (saving every block is now too much almost)
restartSimulationEveryBlocks = 100

# parameters for smc bonds
smcBondWiggleDist = 0.2
smcBondDist = 0.5

# assertions for easy managing code below 
assert (Nframes % restartSimulationEveryBlocks) == 0 
assert (restartSimulationEveryBlocks % saveEveryBlocks) == 0

savesPerSim = restartSimulationEveryBlocks // saveEveryBlocks
simInitsTotal  = (Nframes) // restartSimulationEveryBlocks

## Bond updating functions (for increased speed)

In [3]:
class bondUpdater(object):

    def __init__(self, LEFpositions):
        """
        :param smcTransObject: smc translocator object to work with
        """
        self.LEFpositions = LEFpositions
        self.curtime  = 0
        self.allBonds = []

    def setParams(self, activeParamDict, inactiveParamDict):
        """
        A method to set parameters for bonds.
        It is a separate method because you may want to have a Simulation object already existing

        :param activeParamDict: a dict (argument:value) of addBond arguments for active bonds
        :param inactiveParamDict:  a dict (argument:value) of addBond arguments for inactive bonds

        """
        self.activeParamDict = activeParamDict
        self.inactiveParamDict = inactiveParamDict


    def setup(self, bondForce,  blocks=100, smcStepsPerBlock=1):
        """
        A method that milks smcTranslocator object
        and creates a set of unique bonds, etc.

        :param bondForce: a bondforce object (new after simulation restart!)
        :param blocks: number of blocks to precalculate
        :param smcStepsPerBlock: number of smcTranslocator steps per block
        :return:
        """


        if len(self.allBonds) != 0:
            raise ValueError("Not all bonds were used; {0} sets left".format(len(self.allBonds)))

        self.bondForce = bondForce

        #precalculating all bonds
        allBonds = []
        
        loaded_positions  = self.LEFpositions[self.curtime : self.curtime+blocks]
        allBonds = [[(int(loaded_positions[i, j, 0]), int(loaded_positions[i, j, 1])) 
                        for j in range(loaded_positions.shape[1])] for i in range(blocks)]

        self.allBonds = allBonds
        self.uniqueBonds = list(set(sum(allBonds, [])))

        #adding forces and getting bond indices
        self.bondInds = []
        self.curBonds = allBonds.pop(0)

        for bond in self.uniqueBonds:
            paramset = self.activeParamDict if (bond in self.curBonds) else self.inactiveParamDict
            ind = bondForce.addBond(bond[0], bond[1], **paramset) # changed from addBond
            self.bondInds.append(ind)
        self.bondToInd = {i:j for i,j in zip(self.uniqueBonds, self.bondInds)}
        
        self.curtime += blocks 
        
        return self.curBonds,[]


    def step(self, context, verbose=False):
        """
        Update the bonds to the next step.
        It sets bonds for you automatically!
        :param context:  context
        :return: (current bonds, previous step bonds); just for reference
        """
        if len(self.allBonds) == 0:
            raise ValueError("No bonds left to run; you should restart simulation and run setup  again")

        pastBonds = self.curBonds
        self.curBonds = self.allBonds.pop(0)  # getting current bonds
        bondsRemove = [i for i in pastBonds if i not in self.curBonds]
        bondsAdd = [i for i in self.curBonds if i not in pastBonds]
        bondsStay = [i for i in pastBonds if i in self.curBonds]
        if verbose:
            print("{0} bonds stay, {1} new bonds, {2} bonds removed".format(len(bondsStay),
                                                                            len(bondsAdd), len(bondsRemove)))
        bondsToChange = bondsAdd + bondsRemove
        bondsIsAdd = [True] * len(bondsAdd) + [False] * len(bondsRemove)
        for bond, isAdd in zip(bondsToChange, bondsIsAdd):
            ind = self.bondToInd[bond]
            paramset = self.activeParamDict if isAdd else self.inactiveParamDict
            self.bondForce.setBondParameters(ind, bond[0], bond[1], **paramset)  # actually updating bonds
        self.bondForce.updateParametersInContext(context)  # now run this to update things in the context
        return self.curBonds, pastBonds

## Run simulation 

In [ ]:
milker = bondUpdater(LEFpositions)

reporter = HDF5Reporter(folder=folder, max_data_length=100, overwrite=True, blocks_only=False)

for iteration in range(simInitsTotal):
    
    # simulation parameters are defined below 
    a = Simulation(
            platform="cuda",
            integrator="variableLangevin", 
            error_tol=0.01, 
            GPU = "0", 
            collision_rate=0.03, 
            N = len(data),
            reporters=[reporter],
            PBCbox=[box, box, box],
            precision="mixed")  # timestep not necessary for variableLangevin

    ############################## New code ##############################
    a.set_data(data)  # loads a polymer, puts a center of mass at zero
    
    a.add_force(
        forcekits.polymer_chains(
            a,
            chains=[(0, None, None)],

                # By default the library assumes you have one polymer chain
                # If you want to make it a ring, or more than one chain, use self.setChains
                # self.setChains([(0,50,1),(50,None,0)]) will set a 50-monomer ring and a chain from monomer 50 to the end

            bond_force_func=forces.harmonic_bonds,
            bond_force_kwargs={
                'bondLength':1.0,
                'bondWiggleDistance':0.1, # Bond distance will fluctuate +- 0.05 on average
             },

            angle_force_func=forces.angle_force,
            angle_force_kwargs={
                'k':1.5
                # K is more or less arbitrary, k=4 corresponds to presistence length of 4,
                # k=1.5 is recommended to make polymer realistically flexible; k=8 is very stiff
            },

            nonbonded_force_func=forces.polynomial_repulsive,
            nonbonded_force_kwargs={
                'trunc':1.5, # this will let chains cross sometimes
                'radiusMult':1.05, # this is from old code
                #'trunc':10.0, # this will resolve chain crossings and will not let chain cross anymore
            },

            except_bonds=True,
             
        )
    )
    
    # ------------ initializing milker; adding bonds ---------
    # copied from addBond
    kbond = a.kbondScalingFactor / (smcBondWiggleDist ** 2)
    bondDist = smcBondDist * a.length_scale

    activeParams = {"length":bondDist,"k":kbond}
    inactiveParams = {"length":bondDist, "k":0}
    milker.setParams(activeParams, inactiveParams)
     
    # this step actually puts all bonds in and sets first bonds to be what they should be
    milker.setup(bondForce=a.force_dict['harmonic_bonds'],
                blocks=restartSimulationEveryBlocks)

    # If your simulation does not start, consider using energy minimization below
    if iteration==0:
        a.local_energy_minimization() 
    else:
        a._apply_forces()
    
    for i in range(restartSimulationEveryBlocks):        
        if i % saveEveryBlocks == (saveEveryBlocks - 1):  
            a.do_block(steps=steps)
        else:
            a.integrator.step(steps)  # do steps without getting the positions from the GPU (faster)
        if i < restartSimulationEveryBlocks - 1: 
            curBonds, pastBonds = milker.step(a.context)  # this updates bonds. You can do something with bonds here
    data = a.get_data()  # save data and step, and delete the simulation
    del a
    
    reporter.blocks_only = True  # Write output hdf5-files only for blocks

    time.sleep(0.2)  # wait 200ms for sanity (to let garbage collector do its magic)
    
reporter.dump_data()

INFO:root:Performing local energy minimization
INFO:root:adding force harmonic_bonds 0
INFO:root:adding force angle 1
INFO:root:Using periodic boundary conditions
INFO:root:adding force polynomial_repulsive 2


Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 1.906709
INFO:root:before minimization eK=1.4983582620536646, eP=1.906709104963353, time=0.0 ps
INFO:root:Particles loaded. Potential energy is 0.145231
INFO:root:after minimization eK=1.4983582620536646, eP=0.11359333253342027, time=0.0 ps
INFO:root:block    0 pos[1]=[44.3 6.2 20.3] dr=13.87 t=1319.9ps kin=1.50 pot=1.33 Rg=33.035 dt=176.1fs dx=48.12pm 
INFO:root:block    1 pos[1]=[37.7 8.6 19.8] dr=9.00 t=2640.9ps kin=1.51 pot=1.32 Rg=34.542 dt=176.1fs dx=48.32pm 
INFO:root:block    2 pos[1]=[34.1 14.9 16.5] dr=8.83 t=3961.8ps kin=1.49 pot=1.31 Rg=34.879 dt=176.1fs dx=48.09pm 
INFO:root:block    3 pos[1]=[43.8 17.2 13.8] dr=8.55 t=5282.8ps kin=1.49 pot=1.32 Rg=35.051 dt=176.1fs dx=48.00pm 
INFO:root:block    4 pos[1]=[57.9 16.3 11.5] dr=8.64 t=6603.8ps kin=1.50 pot=1.31 Rg=35.072 dt=176.1fs dx=48.17pm 
INFO:root:block    5 pos[1]=[46.4 20.1 14.9] dr=8.85 t=7924.8ps kin=1.49 pot=1.31 Rg=35.153 dt=176.1fs dx=48.07pm 
INFO:root:block    6 p

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 1.344416
INFO:root:block    0 pos[1]=[36.3 21.2 14.1] dr=8.63 t=1277.1ps kin=1.50 pot=1.32 Rg=35.271 dt=170.3fs dx=46.55pm 
INFO:root:block    1 pos[1]=[36.2 21.8 14.1] dr=8.66 t=2554.2ps kin=1.52 pot=1.32 Rg=35.378 dt=170.3fs dx=46.95pm 
INFO:root:block    2 pos[1]=[35.2 22.4 9.2] dr=8.76 t=3831.3ps kin=1.49 pot=1.31 Rg=35.191 dt=170.3fs dx=46.50pm 
INFO:root:block    3 pos[1]=[38.2 21.1 29.7] dr=8.46 t=5108.4ps kin=1.50 pot=1.31 Rg=35.289 dt=170.3fs dx=46.54pm 
INFO:root:block    4 pos[1]=[33.9 25.5 23.2] dr=8.51 t=6385.5ps kin=1.51 pot=1.31 Rg=35.272 dt=170.3fs dx=46.77pm 
INFO:root:block    5 pos[1]=[24.0 38.0 18.7] dr=8.53 t=7662.6ps kin=1.50 pot=1.31 Rg=35.223 dt=170.3fs dx=46.55pm 
INFO:root:block    6 pos[1]=[32.6 39.1 6.5] dr=8.59 t=8939.7ps kin=1.51 pot=1.31 Rg=35.135 dt=170.3fs dx=46.70pm 
INFO:root:block    7 pos[1]=[34.2 37.5 14.3] dr=8.63 t=10216.8ps kin=1.50 pot=1.32 Rg=35.122 dt=170.3fs dx=46.54pm 
INFO:root:block    8 pos

...

INFO:root:Particles loaded. Potential energy is 1.347809
INFO:root:block    0 pos[1]=[30.0 19.1 13.9] dr=8.64 t=1278.4ps kin=1.51 pot=1.32 Rg=35.297 dt=170.5fs dx=46.73pm 
INFO:root:block    1 pos[1]=[29.6 28.9 14.5] dr=8.53 t=2556.8ps kin=1.50 pot=1.31 Rg=35.209 dt=170.5fs dx=46.61pm 
INFO:root:block    2 pos[1]=[31.7 27.0 18.5] dr=8.48 t=3835.2ps kin=1.49 pot=1.32 Rg=35.293 dt=170.5fs dx=46.51pm 
INFO:root:block    3 pos[1]=[36.0 19.8 12.0] dr=8.61 t=5113.6ps kin=1.50 pot=1.31 Rg=35.301 dt=170.5fs dx=46.57pm 
INFO:root:block    4 pos[1]=[40.9 32.4 16.6] dr=8.58 t=6392.0ps kin=1.50 pot=1.31 Rg=35.145 dt=170.5fs dx=46.66pm 
INFO:root:block    5 pos[1]=[43.7 28.3 1.5] dr=8.41 t=7670.5ps kin=1.50 pot=1.31 Rg=35.187 dt=170.5fs dx=46.65pm 
INFO:root:block    6 pos[1]=[40.7 30.8 6.4] dr=8.55 t=8948.9ps kin=1.50 pot=1.32 Rg=35.205 dt=170.5fs dx=46.69pm 
INFO:root:block    7 pos[1]=[44.6 34.9 8.0] dr=8.60 t=10227.3ps kin=1.51 pot=1.31 Rg=35.245 dt=170.5fs dx=46.74pm 
INFO:root:block    8 pos[

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 1.355619
INFO:root:block    0 pos[1]=[42.8 21.2 -1.7] dr=8.72 t=1274.0ps kin=1.50 pot=1.31 Rg=35.219 dt=169.9fs dx=46.43pm 
INFO:root:block    1 pos[1]=[36.9 22.6 3.2] dr=8.59 t=2547.9ps kin=1.51 pot=1.31 Rg=35.260 dt=169.9fs dx=46.60pm 
INFO:root:block    2 pos[1]=[44.0 17.5 1.8] dr=8.54 t=3821.8ps kin=1.51 pot=1.30 Rg=35.153 dt=169.9fs dx=46.56pm 
INFO:root:block    3 pos[1]=[45.0 20.8 -2.2] dr=8.43 t=5095.7ps kin=1.50 pot=1.31 Rg=35.327 dt=169.9fs dx=46.51pm 
INFO:root:block    4 pos[1]=[34.6 29.7 -6.3] dr=8.67 t=6369.6ps kin=1.48 pot=1.32 Rg=35.298 dt=169.9fs dx=46.22pm 
INFO:root:block    5 pos[1]=[40.2 24.1 -2.5] dr=8.71 t=7643.6ps kin=1.50 pot=1.32 Rg=35.143 dt=169.9fs dx=46.51pm 
INFO:root:block    6 pos[1]=[32.1 24.8 6.6] dr=8.36 t=8917.5ps kin=1.49 pot=1.31 Rg=35.184 dt=169.9fs dx=46.37pm 
INFO:root:block    7 pos[1]=[38.0 22.5 -3.3] dr=8.50 t=10191.4ps kin=1.50 pot=1.31 Rg=35.211 dt=169.9fs dx=46.44pm 
INFO:root:block    8 pos[

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 39999


INFO:root:Particles loaded. Potential energy is 1.350723
INFO:root:block    0 pos[1]=[39.0 34.7 -1.0] dr=8.64 t=1273.8ps kin=1.50 pot=1.32 Rg=35.204 dt=169.8fs dx=46.51pm 
INFO:root:block    1 pos[1]=[35.5 14.8 12.3] dr=8.57 t=2547.5ps kin=1.50 pot=1.31 Rg=35.270 dt=169.8fs dx=46.51pm 
INFO:root:block    2 pos[1]=[39.0 15.5 17.5] dr=8.65 t=3821.3ps kin=1.49 pot=1.31 Rg=35.183 dt=169.8fs dx=46.33pm 
INFO:root:block    3 pos[1]=[44.4 22.4 18.9] dr=8.58 t=5095.0ps kin=1.50 pot=1.31 Rg=35.204 dt=169.8fs dx=46.49pm 
INFO:root:block    4 pos[1]=[35.2 20.8 12.7] dr=8.56 t=6368.8ps kin=1.49 pot=1.32 Rg=35.337 dt=169.8fs dx=46.35pm 


In [ ]:
os.getcwd()